In [1]:
import graphlab as gl
import numpy as np
from datetime import datetime
import math

# Read the datasets and join them

In [2]:
#train = gl.SFrame('./Data/train.csv')

In [3]:
#items = gl.SFrame('./Data/items.csv')
#stores = gl.SFrame('./Data/stores.csv')
#transactions = gl.SFrame("./data/transactions.csv")
#oil = gl.SFrame("./data/oil.csv")
#holidays = gl.SFrame("./data/holidays_events.csv")

In [13]:
#train = train.join(items)
#train = train.join(stores)
#train = train.join(transactions)
#train = train.join(oil)

In [10]:
#train = train[train['dcoilwtico'] != None]

In [61]:
#train['date'] = train['date'].str_to_datetime('%Y-%m-%d')

In [65]:
#train = train.split_datetime('date', limit = {'year', 'month', 'day'})

## Save the joined datasets

In [68]:
#train.save('./data/gl_combined', 'binary')

## Open the saved dataset

In [2]:
train = gl.load_sframe('./data/gl_combined')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1509102040.log


This non-commercial license of GraphLab Create for academic use is assigned to bgj2@hi.is and will expire on July 06, 2018.


In [42]:
train['new_years_ramp'] = 1 / (32 - train['date.day']) * (train['date.month'] == 12)

In [43]:
for i in range(4):
    for feature in ['date.day', 'date.month', 'dcoilwtico', 'new_years_ramp']:
        j = i+2
        name =  feature + "pow" + str(j)
        train[name] = train[feature].apply(lambda x: x**j)

In [44]:
train['month_sin'] = train['date.month'].apply(lambda x: math.sin(x))
train['month_cos'] = train['date.month'].apply(lambda x: math.cos(x))
train['day_sin'] = train['date.day'].apply(lambda x: math.sin(x))
train['day_cos'] = train['date.day'].apply(lambda x: math.cos(x))

In [45]:
typechanges = ['store_nbr', 'item_nbr', 'class', 'perishable', 'cluster']

In [46]:
for var in typechanges:
    train[var] = train[var].astype(str)

In [ ]:
#for var in ['item_nbr', 'store_nbr']:
    #for item in net_train[var].unique():
        #net_train[str(item)] = net_train[var] == str(item)
        #features.append(str(item))

# Train/test split
Með þessu train/test splitti testum við forspánna okkar á 12.júli - 13. ágúst 2017

In [47]:
m = len(train)
cut = int(math.floor(0.97*m))

In [48]:
train_data = train[0:cut]
test_data = train[cut:]

# Quadratic date model

In [46]:
#features = {'date.year', 'date.day', 'date.month', 'date.daypow2', 'date.daypow3',
            #'date.daypow4', 'date.monthpow2', 'date.monthpow3', 'date.monthpow4'}

In [47]:
#date_sqr_model = gl.linear_regression.create(train_data, target = 'unit_sales', features = features)

In [48]:
#date_sqr_model.evaluate(test_data)

# Complex Model

Besta modelið hingað til. Náði 17 validation rmse og 20 test rmse. Er að testa á 

In [50]:
features = {'date.year', 'date.day', 'date.month', 'date.daypow2', 'date.daypow3',
            'date.daypow4', 'date.daypow5', 'date.monthpow2', 'date.monthpow3', 'date.monthpow4',
           'date.monthpow5','month_sin', 'month_cos', 'day_sin', 'day_cos', 'family', 'perishable',
            'class', 'city', 'state', 'cluster', 'onpromotion',
           'dcoilwtico', 'dcoilwticopow2','dcoilwticopow3','dcoilwticopow4','dcoilwticopow5', 
           'new_years_ramp', 'new_years_ramppow2','new_years_ramppow3','new_years_ramppow4',
           'new_years_ramppow5'}

In [ ]:
#for var in ['item_nbr', 'store_nbr']:
    #for item in train_data[var].unique():
        #train_data[str(item)] = net_train[var] == str(item)
        #features.append(str(item))

In [ ]:
complex_model = gl.linear_regression.create(train_data, target = 'unit_sales', features = features,
                                           l1_penalty = 0.01, l2_penalty = 0.02, validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 82346665

Number of features          : 32

Number of unpacked features : 32

...

--------------------------------------------------------

+-----------+----------+-----------+--------------+--------------------+---------------+

| Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+-----------+--------------+--------------------+---------------+

Tuning step size. First iteration could take longer than subsequent iterations.

In [51]:
complex_model.evaluate(test_data)

{'max_error': 7323.907322567533, 'rmse': 20.02763211373246}

# Random Forest

Fyrir menn í miklum anda

In [ ]:
random_forest = gl.random_forest_regression.create(train_data, target = 'unit_sales', features = features)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



External memory mode: 19 batches

Random forest regression:

--------------------------------------------------------

Number of examples          : 78231897

...

Create disk column page 19/19

+-----------+--------------+--------------------+----------------------+---------------+-----------------+

| Iteration | Elapsed Time | Training-max_error | Validation-max_error | Training-rmse | Validation-rmse |

+-----------+--------------+--------------------+----------------------+---------------+-----------------+

| 1         | 575.124460   | 89432.312500       | 8001.310547          | 23.628519     | 19.726761       |

In [29]:
#random_forest.evaluate(test-data)

In [ ]:
random.forest.save('random_forest_model')

# MXNET Neural Network

Þetta er eitthvað sem við getum prófað að keyra til enda á dúndurvél. Mikilvægt að normaliza allar numeric breytur.

In [106]:
norm_features = {'date.year', 'date.day', 'date.month', 'date.daypow2', 'date.daypow3',
            'date.daypow4', 'date.monthpow2', 'date.monthpow3', 'date.monthpow4', 'dcoilwtico'}
for feature in norm_features:
    net_train[feature] = net_train[feature] - net_train[feature].mean()
    net_test[feature] = net_test[feature] - net_train[feature].mean()
    net_train[feature] = net_train[feature] / net_train[feature].std()
    net_test[feature] = net_test[feature] / net_train[feature].std()

In [30]:
#for item in net_train['family'].unique():
    

In [31]:
#net_train = train_data
#net_test = test_data

In [32]:
#from graphlab import mxnet as mx

In [71]:
net = mx.symbol.Variable('data')
net = mx.symbol.FullyConnected(data=net, name='fc1', num_hidden=2)
net = mx.symbol.LinearRegressionOutput(data=net, name='lr')

In [80]:
from graphlab.toolkits.feature_engineering import OneHotEncoder

In [85]:
encoder = gl.feature_engineering.create(net_train, OneHotEncoder(features = ['item_nbr', 'store_nbr']))

In [86]:
transformed_net_train = encoder.transform(net_train)

In [115]:
dataiter_train = mx.io.SFrameIter(net_train, data_field=features, label_field='unit_sales',
                              data_name='data', label_name='lr_label',
                              batch_size=128)

In [ ]:
dataiter_test = mx.io.SFrameIter(net_test, data_field=features, label_field='price',
                              data_name='data', label_name='lr_label',
                              batch_size=128)

In [ ]:
adam = mx.optimizer.Adam()

In [116]:
neural_network = mx.model.FeedForward.create(symbol=net, X=dataiter_train, num_epoch=20,
                                    learning_rate=1e-2, epoch_end_callback = aux_states,
                                    eval_metric='rmse', optimizer = adam)

[INFO] graphlab.mxnet.model: Start training with [cpu(0)]


ToolkitError: Canceled by user

In [ ]:
neural_network.predict(dataiter_train)